In [10]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
import os
import numpy as np
import duckdb
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from collections import defaultdict
from tqdm import tqdm
import copy
#from difflib import SequenceMatcher
#from collections import Counter
#from itertools import chain

In [ ]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [ ]:
con.execute("CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./data/202202-stj/metadadosPublicacao202202.json');")

In [ ]:
con.execute("DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;")

In [ ]:
model_name = "Luciano/xlm-roberta-large-finetuned-lener-br"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=512, chunk_overlap=256
)

In [ ]:
text = """Acórdão de 2018 – Processo n. 1272/14T8SNT.LS1 – Secção.” 3Inequivocamente, inexistiu qualquer alteração ou mudança da configuração da relação jurídica entre as Partes Processo n. 251/11T8CSC.L S1 - 4 Secção Acordam na Secção Social do Supremo Tribunal de Justiça: AA intentou ação declarativa de condenação, sob a forma de processo comum contra COFAC – Cooperativa de Formação e Animação Cultural, CRL, pedindo se condene a R. a: "a) Reconhecer a existência de um contrato de trabalho entre o Autor e a Ré desde 01/10/1999; b)"""

inputs = tokenizer(text, return_tensors="pt")
inputs.input_ids.shape

In [ ]:
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=2)
outputs.logits.shape

In [ ]:
np.array(predictions[0])

In [ ]:
def encontrar_grupos(lista):
    grupos = []
    inicio = None
    fim = None
    for i, valor in enumerate(lista):
        if valor == 11:
            inicio = i
        elif valor == 12 and inicio is not None:
            fim = i
        elif valor != 12 and inicio is not None and fim is not None:
            fim = fim + 1
            grupos.append((inicio, fim))
            inicio = None
            fim = None

    return grupos

In [ ]:
grupos = encontrar_grupos(np.array(predictions[0]))
grupos

In [ ]:
for grupo in grupos:
  print(tokenizer.decode(inputs.input_ids[0][grupo[0]:grupo[1]]))

In [ ]:
model.config.id2label

In [ ]:
entities = [model.config.id2label[prediction] for prediction in predictions[0].tolist()]
print(list(zip(text.split(), entities)))

In [ ]:
data = con.execute("SELECT seqDocumento FROM metadadosPublicacao202202").fetchnumpy()
data["seqDocumento"]

In [ ]:
data["seqDocumento"].size

In [ ]:
file_paths = []  # para armazenar os caminhos completos dos arquivos
file_names = []  # para armazenar apenas os nomes dos arquivos

# caminhando através do diretório raiz e todos os seus subdiretórios
for dirpath, dirnames, filenames in os.walk('./data/202202-stj/', topdown=True):
    for filename in filenames:
        if filename.endswith('.txt'):
            full_path = os.path.join(dirpath, filename)
            file_paths.append(full_path)
            file_names.append(int(filename.split('.')[0]))

In [ ]:
import difflib

def merge_strings(s1, s2, threshold=0.8):
    if len(s2) == 0:
        return s1
    
    s1 = s1.lower()
    s2 = s2.lower()
    
    # Inicializa o SequenceMatcher
    s = difflib.SequenceMatcher(None, s1, s2)

    # Encontra o bloco de correspondência mais longo
    match = s.find_longest_match(0, len(s1), 0, len(s2))
    # Checa se a correspondência é significativa
    if match.size / len(s2) < threshold:
        return s1 + " | " + s2

    # Constrói a nova string usando a parte correspondente de s1 e o restante de s2
    return s1[:match.a] + s2[match.b:]

In [ ]:
jurisprudencias = ['g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'RE 636.', 't no AREsp 1.631.348/', 't no REsp 1.591.422/', 't no REsp 1.591.422/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'RE 636.', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.2014.4.05.', 'JE 0801209-60.2014.4.05.', 'mula 473 do S', 'mula 473 do S', 'RE no 136.236-', 'J 146', 'RE no 136.236-', 'J 146', 'Esp 328', 'Esp 403', 'Esp 328', 'Esp 403', 'são no 1.020/2000-TCU-Plen', 'são no 1.020/2000-TCU-Plen', 'MS 24.859-', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'MS 29.025/', 'g no RE 1.281.817/', 'RE 817.338/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'Processo Administrativo Disciplinar No 23076.000137/2018', 'PE no 1.133.0', 'Processo Administrativo Disciplinar No 23076.000137/2018', 'do de Segurança No 20.16', 'CU no 3088', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.', 'JE 0801209-60.2014.4.05.', 't no REsp 1.551.126/', 't no REsp 1.551.126/', 't no AREsp 1.631.348/', 'mula 473 do Supremo Tribunal', 't no AREsp 1.631.348/', 'mula 473 do Supremo Tribunal', 'mula 473 do Supremo', 'mula 473 do Supremo Tribunal', 'dão TCU 2.161', 'g no REsp 1.551.065/', 'g no REsp 1.499.126/', 'cl nos EDcl no AgRg no REsp 1.285.268/', 'g no REsp 1.563.235/', 'g no REsp 1.563.235/', 'g no REsp. 1.314.843/', 'g no REsp. 1.257.473/', 'g no REsp. 1.166.120/', 'g no Ag.1.116.887/', 'g no Ag 1.342.657/', 'g no Ag 1.297.588/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'do de Segurança 28.2', 'do de Segurança 28.2', 'MS 28.2', 'A 200910000001', 'MS 29.025/', 'MS 29.025/', 'g no RE 1.281.817/', 'aria no 1.104-GM', 'RE 817.338/', 't no REsp 1.442.008/', 't no REsp 1.442.008/', 't no REsp 1.952.026/', 'MULA 7/ST', 't no REsp 1.952.026/', 'MULA 7/ST', 't nos EDcl na AR 6.055/', 't nos EDcl na AR 6.055/', 'mula 7 do ST', 's 7/ST', 't no AREsp 1.163.957/', 't no REsp 1.522.353/', 't no REsp 1.522.353/', 't no RMS 44.511/', 't no RMS 44.511/', 'g no REsp 1400398/', 'g nos EDcl no AgRg no AREsp. 498.224/', 't no REsp 1344578/', 't no REsp 1.667.120/', 't no REsp 1.667.120/']

result = jurisprudencias[0]

for jurisprudencia in jurisprudencias:
  result = merge_strings(result,jurisprudencia)

result

In [ ]:
def process_file(file_path,file_id):
    with open(file_path, 'r') as f:
        content = f.read()
        texts = text_splitter.split_text(content)
        jurisprudencias = np.array([])
        #print(file_id,"texts",len(texts))
        for i,text in enumerate(texts):
            inputs = tokenizer(text, return_tensors="pt")
            #print(file_id,"text",i,inputs.input_ids[0].shape)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=2)
            grupos = encontrar_grupos(np.array(predictions[0]))
            #print(file_id,"jurisprudencia",len(grupos))
            for grupo in grupos:
                jurisprudencia = tokenizer.decode(inputs.input_ids[0][grupo[0]:grupo[1]])
                jurisprudencias = np.append(jurisprudencias, jurisprudencia)
                #print(file_id,jurisprudencia)
        return jurisprudencias

In [ ]:
jurisprudencias = []
for i,doc in tqdm(enumerate(data["seqDocumento"]), total=data["seqDocumento"].size):
  if i < 2000 and i >= 0:  
    if doc in file_names:
      jurisprudencias.append(np.unique(process_file(file_paths[file_names.index(doc)],doc)))

In [2]:
import re

def extrair_numero(texto):
  return re.findall(r'\d+', texto)[0].lstrip('0')
  
def n_sigla(texto):
  text_clear = re.sub(r'\.', '', texto)
  result = re.findall(r'\d*\/[a-z]{2,3}', text_clear)
  if len(result) == 0:
    return text_clear
  else:
    return result[0]

In [3]:
jurisprudencias = pd.read_csv("jurisprudencias0-2000.csv")
jurisprudencias['ref'] = jurisprudencias['ref'].astype(str)
jurisprudencias

,seqDocumento,ref
0,144948780,A 200910000001130
1,144948780,CU no 308898
2,144948780,Esp 328391
3,144948780,Esp 403153
4,144948780,J 146/6
...,...,...
27171,145730312,do sumular n. 182 desta Corte Superior.
27172,145730312,do sumular n. 182 do STJ
27173,145730312,g nos EDcl no AREsp n. 1.785.474/SC
27174,145730312,mula n. 182 do STJ


In [4]:
jurisprudencias = jurisprudencias[jurisprudencias['ref'].str.contains('\d')]
jurisprudencias['ref_numers'] = jurisprudencias['ref'].apply(lambda x: extrair_numero(x))
jurisprudencias = jurisprudencias.drop_duplicates(subset="ref_numers", keep='first')
jurisprudencias['termo'] = jurisprudencias['ref'].str.lower().apply(lambda x: x+" "+n_sigla(x)+(" "+extrair_numero(x))*2)

jurisprudencias

/tmp/ipykernel_39989/1664226564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jurisprudencias['ref_numers'] = jurisprudencias['ref'].apply(lambda x: extrair_numero(x))


,seqDocumento,ref,ref_numers,termo
0,144948780,A 200910000001130,200910000001130,a 200910000001130 a 200910000001130 2009100000...
1,144948780,CU no 308898,308898,cu no 308898 cu no 308898 308898 308898
2,144948780,Esp 328391,328391,esp 328391 esp 328391 328391 328391
3,144948780,Esp 403153,403153,esp 403153 esp 403153 403153 403153
4,144948780,J 146/6,146,j 146/6 j 146/6 146 146
...,...,...,...,...
27113,145564564,t no REsp 1827460/MG,1827460,t no resp 1827460/mg 1827460/mg 1827460 1827460
27120,146032219,t no AREsp 1224156/SP,1224156,t no aresp 1224156/sp 1224156/sp 1224156 1224156
27121,146032219,t no AREsp 1317073/MS,1317073,t no aresp 1317073/ms 1317073/ms 1317073 1317073
27143,145495676,t no AREsp 1084864/SP,1084864,t no aresp 1084864/sp 1084864/sp 1084864 1084864


In [5]:
resultado = jurisprudencias[jurisprudencias['ref_numers'] == '7']
resultado

,seqDocumento,ref,ref_numers,termo
11,144948780,MULA 7/STJ,7,mula 7/stj 7/stj 7 7


In [ ]:
con.execute("CREATE TABLE refs(id INT, data TEXT);")
con.execute("CREATE INDEX refs_index ON refs (id,data);")

In [6]:
import textdistance

jaro_winkler = textdistance.JaroWinkler(external=True)
damerau_levenshtein = textdistance.DamerauLevenshtein(external=True)
def similar(s1, s2):
    #sequence_matcher = SequenceMatcher(None, s1.lower(), s2.lower()).ratio() # 2957-2:16:09
    return damerau_levenshtein.normalized_similarity(s1, s2)
    #jaro_winkler_sim = textdistance.jaro_winkler.normalized_similarity(s1.lower(), s2.lower())
    #return jaro_winkler.normalized_similarity(s1, s2)
    #return jaro_winkler.normalized_distance(s1, s2)

In [147]:
#similar("turma","turma")
#similar("hc 598.051/sp da sexta turma",". 598.051/sp") #"hc 598.051/"
similar("esp 1785474/sc "+"1785474/sc "*8,resultado.termo.values[0])

0.9196428571428571

In [148]:
similar("esp 1785474/sc","esp 111785474/sc")

0.875

In [149]:
similar("esp 1785474/sc","esp 1785472/sc")

0.9285714285714286

In [7]:
ref_unicos = jurisprudencias['termo'].unique()
ref_unicos.size

2831

In [ ]:
con.execute("""SELECT * FROM refs WHERE data = 'JE 0801209-60.2014' """).df()

In [8]:
import statistics

grups_refs = [[ref_unicos[0]]]

# para saber e ref pertence ao grupo calcular similadridade media de todos os elemtos do grupo

for i,ref in tqdm(enumerate(ref_unicos), total=len(ref_unicos)):
  for j,grup_refs in enumerate(grups_refs):
    similar_refs = []
    for k in range(len(grup_refs)): 
      similar_refs.append(similar(ref, grup_refs[k]))

    media = statistics.mean(similar_refs)
    if media > 0.80:
      grups_refs[j].append(ref)
      break

  grups_refs.append([ref])

  0%|          | 0/2899 [00:00<?, ?it/s]

100%|██████████| 2899/2899 [04:43<00:00, 10.23it/s]


In [9]:
grups_refs_norm = copy.deepcopy(grups_refs)
for i,grup_refs in enumerate(grups_refs_norm):
  for j,ref in enumerate(grup_refs):
    grups_refs_norm[i][j] = jurisprudencias[jurisprudencias['termo'] == ref].ref.values[0]

In [55]:
len(grups_refs_norm)

2832

In [8]:
grups_refs = [ref_unicos[0]]

for i,ref in tqdm(enumerate(ref_unicos), total=len(ref_unicos)):
  for j,grup_refs in enumerate(grups_refs):
    similar_refs = similar(grup_refs,ref)

    if similar_refs > 0.85:
      grups_refs[j] = grup_refs + " | " + ref
      break

  grups_refs.append(ref)

100%|██████████| 2831/2831 [01:05<00:00, 43.40it/s] 


In [11]:
grups_refs_norm = copy.deepcopy(grups_refs)
for i,grup_refs in enumerate(grups_refs_norm):
  grups_refs_norm[i] = grup_refs.split(" | ")
  
for i,grup_refs in enumerate(grups_refs_norm):
  for j,ref in enumerate(grup_refs):
    grups_refs_norm[i][j] = jurisprudencias[jurisprudencias['termo'] == ref].ref.values[0]

In [ ]:
import json

with open('grups_refs.json', 'w') as arquivo_json:
    json.dump(grups_refs_norm, arquivo_json)

In [ ]:
# Abra o arquivo JSON em modo de leitura
with open('grups_refs_jaro_winkler.json', 'r') as arquivo:
    # Use a função json.load() para carregar o conteúdo do arquivo JSON em um objeto Python
    grups_refs = json.load(arquivo)

In [ ]:
#for i,doc in tqdm(enumerate(jurisprudencias['ref']), total=ref_unicos.size):
for i,doc in enumerate(jurisprudencias['ref']):
  print(i,doc)

In [ ]:
# Aplanando a lista de listas
#lista_aplanada = list(chain.from_iterable(jurisprudencias))

lista_aplanda = jurisprudencias['ref']

contagem = defaultdict(int)

for i in tqdm(range(len(lista_aplanda))):
    for j in range(i, len(lista_aplanda)):
        #if similar(lista_aplanda[i], lista_aplanda[j]):
        if lista_aplanda[i] == lista_aplanda[j]:
            contagem[lista_aplanda[i]] += 1

contagem

In [ ]:
jurisprudencias['new_column'] = jurisprudencias['ref'].map(contagem)
jurisprudencias

In [ ]:
jurisprudencias = jurisprudencias.sort_values(by='new_column', ascending=False)
jurisprudencias.to_csv('contagem_s_jurisprudencia.csv', index=False)
jurisprudencias

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('pt_core_news_lg')

In [ ]:
doc = nlp("Exemplo de texto com algumas stopwords")
filtered_sentence = [token.text for token in doc if not token.is_stop]
" ".join(filtered_sentence)

In [ ]:
def process_file(file_path,file_id):
    with open(file_path, 'r') as f:
        content = f.read()
        doc = nlp(content)
        content = " ".join([token.text for token in doc if not token.is_stop])
        texts = text_splitter.split_text(content)
        jurisprudencias = np.array([])
        print(file_id,"texts",len(texts))
        for i,text in enumerate(texts):
            inputs = tokenizer(text, return_tensors="pt")
            #print(file_id,"text",i,inputs.input_ids[0].shape)
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=2)
            grupos = encontrar_grupos(np.array(predictions[0]))
            #print(file_id,"jurisprudencia",len(grupos))
            for grupo in grupos:
                jurisprudencia = tokenizer.decode(inputs.input_ids[0][grupo[0]:grupo[1]])
                jurisprudencias = np.append(jurisprudencias, jurisprudencia)
                #print(file_id,jurisprudencia)
        return jurisprudencias

In [ ]:
jurisprudencias = np.array([])
for i,doc in enumerate(data["seqDocumento"]):
  if i < 10:  
    if doc in file_names:
      jurisprudencias = np.concatenate((jurisprudencias, process_file(file_paths[file_names.index(doc)],doc)))

In [ ]:
jurisprudencias = np.unique(jurisprudencias)
"|".join(jurisprudencias)

In [ ]:
con.commit()

In [ ]:
con.close()